Kflod spliting keeping entries of the same image at the same fold

In [ ]:
from sklearn import model_selection
import pandas as pd

DIR_INPUT = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'
train_df = pd.read_csv(f'{DIR_INPUT}/train.csv')

train_df.fillna(0, inplace=True)
train_df.loc[train_df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

unique_images = train_df["image_id"].unique()
df_split = pd.DataFrame(unique_images, columns = ['unique_images']) 



df_split["kfold"] = -1
df_split = df_split.sample(frac=1).reset_index(drop=True)
y = df_split.unique_images.values

kf = model_selection.GroupKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df_split, y=y, groups=df_split.unique_images.values)):
    df_split.loc[v_, "kfold"] = f
df_split
# For now during training we are feeding an image with all the anotated boxes of that image as one input to the model.
# So makes sense to have same "image id" should have same fold.

train_df["kfold"] = -1
for ind in train_df.index: 
     train_df["kfold"][ind] = df_split.loc[ df_split["unique_images"] ==  train_df["image_id"][ind]]["kfold"]
    

train_df.set_index('image_id', inplace=True)
train_df